# Notebook 6: Agent Semplice con Tools Base

**Obiettivo**: Implementare un Agent ReAct base con tools personalizzati usando LangChain

---


## 1. Setup e Import

Importiamo tutte le librerie necessarie per Agent, Tools e LLM.

Questo notebook usa `create_agent` di LangChain 1.2+.
In LangChain 1.2+, `create_agent` restituisce un `StateGraph` che gestisce automaticamente il tool calling loop.


In [ ]:
# LangChain 1.2+ (API moderna con create_agent)
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

# Utilità
import math
from datetime import datetime

# Inizializza LLM (chat model per Agent)
# NOTA: Agent richiedono chat model con supporto tool calling
# ChatOllama supporta tool calling nativo in LangChain 1.0+
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0.7
)

print("✅ Setup completato!")
print(f"LLM: llama3.2:3b (ChatOllama)")
print(f"API: LangChain 1.2+ (create_agent)")


## 2. Creare Tools Personalizzati

Un Tool è una funzione che l'Agent può chiamare. Usiamo il decorator `@tool` per creare tools facilmente.


In [ ]:
# Tool 1: Calcolatrice
@tool
def calculator(expression: str) -> str:
    """Esegue calcoli matematici.
    
    Args:
        expression: Espressione matematica come stringa (es. "2 + 2", "10 * 5", "sqrt(16)")
    
    Returns:
        Risultato del calcolo come stringa
    
    Esempi:
        - "2 + 2" → "4"
        - "10 * 5" → "50"
        - "sqrt(16)" → "4.0"
    """
    try:
        # ⚠️ In produzione, usa un parser matematico sicuro (es. sympy)
        # Qui usiamo eval() solo per semplicità didattica
        result = eval(expression, {"__builtins__": {}}, {"math": math, "sqrt": math.sqrt})
        return str(result)
    except Exception as e:
        return f"Errore nel calcolo: {str(e)}"

# Tool 2: Conversione unità
@tool
def convert_units(value: float, from_unit: str, to_unit: str) -> str:
    """Converte valori tra unità di misura.
    
    Args:
        value: Valore da convertire
        from_unit: Unità di origine (es. "km", "kg", "celsius")
        to_unit: Unità di destinazione (es. "m", "g", "fahrenheit")
    
    Returns:
        Valore convertito come stringa
    
    Esempi:
        - convert_units(1, "km", "m") → "1000.0 m"
        - convert_units(100, "celsius", "fahrenheit") → "212.0 fahrenheit"
    """
    conversions = {
        # Lunghezza
        ("km", "m"): lambda x: x * 1000,
        ("m", "km"): lambda x: x / 1000,
        ("m", "cm"): lambda x: x * 100,
        ("cm", "m"): lambda x: x / 100,
        # Peso
        ("kg", "g"): lambda x: x * 1000,
        ("g", "kg"): lambda x: x / 1000,
        # Temperatura
        ("celsius", "fahrenheit"): lambda x: (x * 9/5) + 32,
        ("fahrenheit", "celsius"): lambda x: (x - 32) * 5/9,
    }
    
    key = (from_unit.lower(), to_unit.lower())
    if key in conversions:
        result = conversions[key](value)
        return f"{result} {to_unit}"
    else:
        return f"Conversione non supportata: {from_unit} → {to_unit}"

# Tool 3: Informazioni data/ora
@tool
def get_current_time(format: str = "standard") -> str:
    """Restituisce la data e ora corrente.
    
    Args:
        format: Formato richiesto ("standard", "iso", "timestamp")
    
    Returns:
        Data e ora formattata
    """
    now = datetime.now()
    
    if format == "iso":
        return now.isoformat()
    elif format == "timestamp":
        return str(now.timestamp())
    else:  # standard
        return now.strftime("%Y-%m-%d %H:%M:%S")

# Tool 4: Ricerca web (mock - per demo)
@tool
def search_web(query: str) -> str:
    """Cerca informazioni sul web (MOCK - per demo).
    
    In produzione, questo tool chiamerebbe un'API di ricerca reale (Google, Bing, etc.)
    
    Args:
        query: Query di ricerca
    
    Returns:
        Risultati ricerca (mock)
    """
    # Mock results per demo
    mock_results = {
        "python": "Python è un linguaggio di programmazione ad alto livello...",
        "langchain": "LangChain è un framework per sviluppare applicazioni con LLM...",
        "ollama": "Ollama è uno strumento per eseguire LLM localmente...",
    }
    
    query_lower = query.lower()
    for key, value in mock_results.items():
        if key in query_lower:
            return f"Risultati per '{query}': {value}"
    
    return f"Risultati per '{query}': Informazioni non trovate nel database mock."

print("✅ Tools creati:")
print("  1. calculator - Calcoli matematici")
print("  2. convert_units - Conversione unità")
print("  3. get_current_time - Data/ora corrente")
print("  4. search_web - Ricerca web (mock)")


## 3. Test Tools Individualmente

Prima di creare l'Agent, testiamo che i tools funzionino correttamente.


In [ ]:
# Test calcolatrice
result1 = calculator.invoke("2 + 2")
print(f"Calcolatrice: 2 + 2 = {result1}")

# Test conversione unità
result2 = convert_units.invoke({"value": 1, "from_unit": "km", "to_unit": "m"})
print(f"Conversione: 1 km = {result2}")

# Test data/ora
result3 = get_current_time.invoke({"format": "standard"})
print(f"Data/ora: {result3}")

# Test ricerca web
result4 = search_web.invoke("python")
print(f"Ricerca: {result4}")

print("\n✅ Tutti i tools funzionano correttamente!")


## 4. Creare Agent ReAct

Creiamo un Agent ReAct che può usare i tools. L'Agent decide automaticamente quale tool chiamare e quando.


In [ ]:
# Raccogli tutti i tools in una lista
tools = [calculator, convert_units, get_current_time, search_web]

# Crea Agent con create_agent (LangChain 1.2+)
# create_agent crea un StateGraph che gestisce automaticamente tool calling
# Il system_prompt guida il comportamento dell'Agent
system_prompt = """Sei un assistente utile che può usare tools per rispondere alle domande.

Hai accesso ai seguenti tools:
- calculator: Per calcoli matematici
- convert_units: Per convertire unità di misura
- get_current_time: Per ottenere data e ora corrente
- search_web: Per cercare informazioni sul web

Usa i tools quando necessario per rispondere alle domande dell'utente.
Se non serve un tool, rispondi direttamente.

IMPORTANTE:
- Usa sempre i tools quando appropriato
- Fornisci risposte chiare e complete
- Se un tool fallisce, prova un approccio alternativo"""

# Crea Agent (restituisce un CompiledStateGraph)
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    debug=True  # Mostra ragionamento Agent
)

print("✅ Agent creato con create_agent (LangChain 1.2+)!")
print(f"Tools disponibili: {len(tools)}")
print(f"Agent type: StateGraph (gestisce automaticamente tool calling loop)")


## 5. Test Agent con Query Semplici

Testiamo l'Agent con query che richiedono un singolo tool.


In [ ]:
# Test 1: Calcolo matematico
query1 = "Quanto fa 15 * 23?"
print(f"Query: {query1}\n")

# In LangChain, l'agent è un StateGraph che si invoca direttamente
# L'input è un dizionario con "messages" contenente la query
result1 = agent.invoke({"messages": [HumanMessage(content=query1)]})
print(f"\nRisposta: {result1['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 2: Conversione unità
query2 = "Converti 5 chilometri in metri"
print(f"Query: {query2}\n")
result2 = agent.invoke({"messages": [HumanMessage(content=query2)]})
print(f"\nRisposta: {result2['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 3: Data/ora
query3 = "Che ore sono?"
print(f"Query: {query3}\n")
result3 = agent.invoke({"messages": [HumanMessage(content=query3)]})
print(f"\nRisposta: {result3['messages'][-1].content}\n")
print("="*60)


## 6. Test Agent con Query Complesse (Multi-Tool)

Testiamo l'Agent con query che richiedono multiple tools o ragionamento complesso.


In [ ]:
# Test 4: Query che richiede calcolo + conversione
query4 = "Quanto sono 10 miglia in metri? (1 miglio = 1.609 km)"
print(f"Query: {query4}\n")
result4 = agent.invoke({"messages": [HumanMessage(content=query4)]})
print(f"\nRisposta: {result4['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 5: Query che richiede ricerca + calcolo
query5 = "Cerca informazioni su Python e poi calcola la radice quadrata di 144"
print(f"Query: {query5}\n")
result5 = agent.invoke({"messages": [HumanMessage(content=query5)]})
print(f"\nRisposta: {result5['messages'][-1].content}\n")
print("="*60)


## 7. Debugging Agent (Verbose Mode)

Con `verbose=True`, possiamo vedere il ragionamento dell'Agent step-by-step. Questo è utile per debugging.


In [ ]:
# Query di esempio per vedere verbose output
query_debug = "Calcola 25 * 4 e poi converti il risultato da metri a chilometri"

print("=== DEBUG MODE: Ragionamento Agent ===\n")
# Con debug=True, l'agent mostra i passaggi interni
result_debug = agent.invoke({"messages": [HumanMessage(content=query_debug)]})

print("\n=== Risultato Finale ===")
print(result_debug['messages'][-1].content)

print("\n💡 Con debug=True, l'agent mostra:")
print("  - Messaggi intermedi (tool calls, responses)")
print("  - State transitions")
print("  - Risposta finale")


## 8. Gestione Errori e Fallback

L'Agent può gestire errori e provare approcci alternativi.


In [ ]:
# Test con query che potrebbe causare errore
query_error = "Calcola la radice quadrata di -16"

print(f"Query: {query_error}\n")
result_error = agent.invoke({"messages": [HumanMessage(content=query_error)]})
print(f"\nRisposta: {result_error['messages'][-1].content}\n")

print("💡 L'Agent gestisce errori e fornisce risposte utili anche quando i tools falliscono.")


## 9. Note e Best Practices

### Cosa abbiamo imparato:
1. **Tools**: Funzioni che Agent può chiamare usando `@tool` decorator
2. **create_agent**: API moderna LangChain 1.2+ che crea StateGraph
3. **StateGraph**: Gestisce automaticamente tool calling loop
4. **Multi-Tool**: Agent può usare multiple tools in sequenza
5. **Debug Mode**: Utile per vedere ragionamento Agent
6. **Error Handling**: Agent gestisce errori e fallback

### Best Practices:

**Tools**:
- Descrizioni chiare: Agent usa descrizioni per decidere quale tool chiamare
- Error handling: Tools dovrebbero gestire errori e restituire messaggi utili
- Validazione input: Valida input prima di processare

**Agent**:
- System prompt: Prompt chiaro migliora performance Agent
- Debug mode: Attiva in sviluppo (`debug=True`), disattiva in produzione
- StateGraph: Gestisce automaticamente loop e stopping conditions

**Debugging**:
- Testare i tools individualmente prima di usarli con un Agent
- Usare la verbose mode per capire ragionamento dell'Agent
- Verificare che i tools abbiano descrizioni chiare

### Limitazioni e Considerazioni:
- **Performance**: Gli Agent sono più lenti dei chatbot semplici (molte chiamate LLM)
- **Costo**: Gli Agent consumano più token (costo maggiore se cloud)
- **Loop infiniti**: Gli Agent possono entrare in loop (usare max_iterations)
- **Tool selection**: Un Agent può scegliere tool sbagliato (migliorare descrizioni)

---

**Congratulazioni! Hai completato il Notebook 6! 🎉**
